
# Complex seismic processing workflow: **SAC (via `subprocess`)** vs **ObsPy**

This notebook demonstrates a **typical 3‑component processing chain** in two ways:

1. **SAC** is driven non-interactively from Python using `subprocess` and a multi-line SAC command string (a pattern shown in the IRIS SAC manual section on calling SAC from scripts).  
2. **ObsPy** performs the same processing steps in pure Python.

Workflow steps (both implementations):
- read 3 components (Z, N, E) for a station/event
- verify consistent start/end times and sampling
- **demean + linear detrend**
- **taper**
- **high-pass filter**
- **remove instrument response to velocity**
- compute **3‑D PGV**:  \( \max_t \sqrt{v_Z(t)^2 + v_N(t)^2 + v_E(t)^2} \)

---

## Assumptions / inputs you must provide

You need:
- three waveform files (SAC or MiniSEED) for the same time window:
  - `*_BHZ.*`, `*_BHN.*`, `*_BHE.*` (change patterns below if needed)
- instrument response metadata:
  - **SAC path**: RESP files (EvalResp) **or** PoleZero files, plus SAC headers populated with NET/STA/LOC/CHA
  - **ObsPy path**: `StationXML` (preferred)

The cells below are written to be robust: they check whether `sac` exists, whether files exist, and they fail with clear messages.

> IRIS SAC scripting reference:
> https://ds.iris.edu/files/sac-manual/manual/sac_script.html


## Configuration (shared by both approaches)

In [ ]:

from pathlib import Path

# --- Input selection ---
# Use explicit filenames when possible; globs are convenient but can match unintended files.
Z_FILE = None   # e.g., "XX.STA..BHZ.SAC"
N_FILE = None   # e.g., "XX.STA..BHN.SAC"
E_FILE = None   # e.g., "XX.STA..BHE.SAC"

# If you prefer globbing, set these patterns and keep *_FILE = None
Z_GLOB = "*BHZ*.SAC"
N_GLOB = "*BHN*.SAC"
E_GLOB = "*BHE*.SAC"

# --- Output directory ---
OUT_DIR = Path("processed")
OUT_DIR.mkdir(exist_ok=True)

# --- Processing parameters ---
HP_CORNER_HZ = 0.5      # high-pass corner (Hz)
CORNERS = 4             # filter order (SAC: n 4)
ZEROPHASE = True        # SAC: p 2 (two-pass zero-phase)
TAPER_PCT = 0.05

# --- Instrument response removal ---
# SAC approach (EvalResp):
# Option A: let SAC find RESP via SAC headers (NET/STA/LOC/CHA) if your setup supports it.
# Option B: provide explicit RESP filenames below.
RESP_Z = None  # e.g., "RESP.XX.STA..BHZ"
RESP_N = None  # e.g., "RESP.XX.STA..BHN"
RESP_E = None  # e.g., "RESP.XX.STA..BHE"

# Safe deconvolution frequency limits (SAC "freqlimits f1 f2 f3 f4")
F1, F2, F3, F4 = 0.01, 0.02, 20.0, 25.0

def resolve_one(explicit, pattern, label):
    if explicit:
        p = Path(explicit)
        if not p.exists():
            raise FileNotFoundError(f"{label} file not found: {p}")
        return p
    matches = sorted(Path('.').glob(pattern))
    if len(matches) == 0:
        raise FileNotFoundError(f"No files match {label} glob: {pattern!r}")
    if len(matches) > 1:
        raise RuntimeError(
            f"Ambiguous {label} glob {pattern!r}. Matches: {[m.name for m in matches]}. "
            f"Set {label}_FILE explicitly."
        )
    return matches[0]

z_path = resolve_one(Z_FILE, Z_GLOB, "Z")
n_path = resolve_one(N_FILE, N_GLOB, "N")
e_path = resolve_one(E_FILE, E_GLOB, "E")

z_path, n_path, e_path



## SAC via `subprocess` (single SAC command block)

This cell:
- builds one **multi-line SAC script string**
- feeds it to `sac` via `subprocess.Popen(...).communicate(...)` (as shown in the IRIS SAC scripting page)
- writes processed velocity SAC files
- computes **3‑D PGV inside SAC** by:
  1) squaring each component (`sqr`)  
  2) summing squares into a master trace using `addf master 0`  
  3) square-rooting (`sqrt`) to get vector magnitude  
  4) reporting `max` of that magnitude trace  
- stores PGV in `user0` on the magnitude trace (`chnhdr user0 &1,max&`)

> If your `transfer from evalresp` cannot find responses automatically, set `RESP_Z/RESP_N/RESP_E` explicitly.


In [ ]:

import os
import re
import subprocess

def require_executable(name: str):
    from shutil import which
    if which(name) is None:
        raise EnvironmentError(
            f"Required executable {name!r} not found on PATH. "
            f"Install/configure it and try again."
        )

require_executable("sac")

# Optional: suppress SAC copyright banner (per IRIS scripting doc)
env = os.environ.copy()
env["SAC_DISPLAY_COPYRIGHT"] = "0"

# Outputs
out_z = OUT_DIR / (z_path.stem + ".VEL.SAC")
out_n = OUT_DIR / (n_path.stem + ".VEL.SAC")
out_e = OUT_DIR / (e_path.stem + ".VEL.SAC")
out_mag = OUT_DIR / (z_path.stem.replace("BHZ", "VMAG") + ".SAC")

def transfer_cmd(resp):
    # If resp is provided, we tell SAC exactly which RESP file to use.
    # If not, we try without FNAME (works in some environments if headers + RESP search path are set).
    if resp:
        return f"transfer from evalresp fname {resp} to vel freqlimits {F1} {F2} {F3} {F4}"
    return f"transfer from evalresp to vel freqlimits {F1} {F2} {F3} {F4}"

# One SAC command block (processed sequentially inside one SAC session)
sac_script = f"""echo on
wild echo off

* ---- Z to velocity ----
read {z_path}
rmean
rtrend
taper
hp co {HP_CORNER_HZ} n {CORNERS} p 2
{transfer_cmd(RESP_Z)}
write over {out_z}

* ---- N to velocity ----
read {n_path}
rmean
rtrend
taper
hp co {HP_CORNER_HZ} n {CORNERS} p 2
{transfer_cmd(RESP_N)}
write over {out_n}

* ---- E to velocity ----
read {e_path}
rmean
rtrend
taper
hp co {HP_CORNER_HZ} n {CORNERS} p 2
{transfer_cmd(RESP_E)}
write over {out_e}

* ---- 3-D PGV ----
read {out_z} {out_n} {out_e}
sqr
addf master 0
sqrt
write over {out_mag}

max
chnhdr user0 &1,max&
write over {out_mag}

quit
"""

p = subprocess.Popen(
    ["sac"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    env=env,
    text=True
)

stdout = p.communicate(sac_script)[0]
print(stdout)

m = re.search(r"MAXIMUM AMPLITUDE\s*=\s*([0-9Ee+\-\.]+)", stdout)
if m:
    pgv = float(m.group(1))
    print(f"Parsed PGV (3-D vector) from SAC output: {pgv:g} (m/s if response removal produced velocity)")
else:
    print("Could not parse PGV from SAC output. Check the 'max' output above.")



## ObsPy (single Python processing block)

This cell mirrors the same workflow but entirely in ObsPy:

- reads Z/N/E
- trims to common overlap (important for vector magnitude)
- detrend + taper
- high-pass
- remove instrument response to **velocity**
- computes 3‑D PGV
- writes processed SAC outputs (velocity components + magnitude)

Replace `station.xml` with your StationXML path.


In [ ]:

import numpy as np
from obspy import read, read_inventory, Trace

# --- Read waveforms ---
st = read(str(z_path)) + read(str(n_path)) + read(str(e_path))

# Select components (adjust if needed)
trZ = st.select(channel="*Z")[0]
trN = st.select(channel="*N")[0]
trE = st.select(channel="*E")[0]

# --- Common overlap window ---
t_start = max(trZ.stats.starttime, trN.stats.starttime, trE.stats.starttime)
t_end   = min(trZ.stats.endtime,   trN.stats.endtime,   trE.stats.endtime)
if t_end <= t_start:
    raise RuntimeError("No overlapping time window across Z/N/E traces.")

for tr in (trZ, trN, trE):
    tr.trim(t_start, t_end, pad=False)

# --- Sample interval check ---
deltas = {tr.stats.delta for tr in (trZ, trN, trE)}
if len(deltas) != 1:
    raise RuntimeError(f"Sample intervals differ across components: {deltas}. Resample first.")

# --- Preprocess ---
for tr in (trZ, trN, trE):
    tr.detrend("demean")
    tr.detrend("linear")
    tr.taper(max_percentage=TAPER_PCT)
    tr.filter("highpass", freq=HP_CORNER_HZ, corners=CORNERS, zerophase=ZEROPHASE)

# --- Response removal to velocity ---
inv = read_inventory("station.xml")
for tr in (trZ, trN, trE):
    tr.remove_response(
        inventory=inv,
        output="VEL",
        pre_filt=(F1, F2, F3, F4),
        water_level=60
    )

# --- 3-D PGV ---
vZ = trZ.data.astype(float)
vN = trN.data.astype(float)
vE = trE.data.astype(float)

vmag = np.sqrt(vZ*vZ + vN*vN + vE*vE)
pgv_3d = float(np.nanmax(np.abs(vmag)))
i_max = int(np.nanargmax(np.abs(vmag)))
t_pgv = trZ.stats.starttime + i_max * trZ.stats.delta

print(f"3-D PGV = {pgv_3d:g} m/s at {t_pgv}")

# --- Write outputs ---
outZ = OUT_DIR / (z_path.stem + ".VEL.OBSPY.SAC")
outN = OUT_DIR / (n_path.stem + ".VEL.OBSPY.SAC")
outE = OUT_DIR / (e_path.stem + ".VEL.OBSPY.SAC")

trZ.write(str(outZ), format="SAC")
trN.write(str(outN), format="SAC")
trE.write(str(outE), format="SAC")

trMag = Trace(data=vmag.astype(np.float32), header=trZ.stats)
trMag.stats.channel = trZ.stats.channel[:-1] + "M"  # e.g., BHZ -> BHM
outMag = OUT_DIR / (z_path.stem.replace("BHZ", "VMAG") + ".OBSPY.SAC")
trMag.write(str(outMag), format="SAC")

print("Wrote processed outputs to:", OUT_DIR.resolve())
